In [11]:
import pandas as pd
import os
import regex as re

In [12]:
os.getcwd()

'/mnt/raidinput2/tmp/joppich/pubmed_pmc/pmc/vimtm/covidsents'

In [13]:
sheetsDF = pd.read_excel("PMC8078050/PMC8078050_mmc5.xlsx", header=None, sheet_name=None)
print([x for x in sheetsDF])

['Vmir outputs', 'Maturebayes inputs', 'Maturebays_outputs', 'mature mirnas (5 prime)', 'miRNAs fasta format']


In [14]:
df = sheetsDF['miRNAs fasta format']
df

,0
0,>SARS-CoV-miR001
1,CGTGCTAGTGCGTGGCTTCGGG
2,>SARS-CoV-miR002
3,TAAAGTCTTATGACTTAGGTGA
4,>SARS-CoV-miR003
...,...
269,TTTTGGCAATGTTGTTCCTTGA
270,>SARS-CoV-miR136
271,AGAGTAGACTATGTATCGTAAA
272,>SARS-CoV-miR137


In [15]:
fastaStartRE = re.compile(r'^>\S\S\S\S+')
fastaSequenceMirRE = re.compile(r'^[ATCGU]{15,30}$')

for sheet in sheetsDF:

    df = sheetsDF[sheet]

    for column in df.columns:

        bestRowSequence = None
        rowSequence = []
        
        for rowIdx, row in df.iterrows():

            fastaStart = fastaStartRE.match(str(row[column]))
            fastaSequenceMir = fastaSequenceMirRE.match(str(row[column]))

            if fastaStart:
                #print(rowIdx, "fasta start")
                rowSequence.append("start")
            if fastaSequenceMir:
                #print(rowIdx, "fasta seq")
                rowSequence.append("seq")

            if not fastaStart and not fastaSequenceMir:
                if len(rowSequence) > 0 and (bestRowSequence == None or len(bestRowSequence) > len(rowSequence)):
                    bestRowSequence = rowSequence

            if rowIdx > 20:
                break
        if len(rowSequence) > 0 and (bestRowSequence == None or len(bestRowSequence) > len(rowSequence)):
            bestRowSequence = rowSequence

        if not bestRowSequence is None:

            count = 0
            bestCount = 0
            for i in range(1, len(bestRowSequence)):

                if bestRowSequence[i-1] != bestRowSequence[i]:
                    count += 1

                else:
                    if count > bestCount:
                        bestCount = count

            if count > bestCount:
                        bestCount = count

            if bestCount > 0:
                print(sheet, column, bestRowSequence)
                print(sheet, column, bestCount)

miRNAs fasta format 0 ['start', 'seq', 'start', 'seq', 'start', 'seq', 'start', 'seq', 'start', 'seq', 'start', 'seq', 'start', 'seq', 'start', 'seq', 'start', 'seq', 'start', 'seq', 'start', 'seq']
miRNAs fasta format 0 21


In [16]:
len("ATTATCAAGGTCTCCCTCTGGAATTTGGTGCCTCAGCTGAAACAGTTCGAGTTGAGGAAGAAGAAGAGGAAGACTGGCTGGATGAT")

86